# В этой домашке мы поработаем с эмбеддингами текстов и сделаем первое соревнование по нлп

## Начнем с подготовки (0.5 баллов)

Создадим "голову", которая будет отвечать за классификацию на основе эмбеддингов. Для этого обучим CatBoost на заранее подготовленных эмбеддингах с добавленным шумом.

In [1]:
import pandas as pd

train = pd.read_csv('/kaggle/input/fake-data-hw1/FAKE_train.csv').drop(columns=['Unnamed: 0'])
test = pd.read_csv('/kaggle/input/fake-data-hw1/FAKE_test.csv').drop(columns=['Unnamed: 0'])
train

,bert_dim_0,bert_dim_1,bert_dim_2,bert_dim_3,bert_dim_4,bert_dim_5,bert_dim_6,bert_dim_7,bert_dim_8,bert_dim_9,...,bert_dim_759,bert_dim_760,bert_dim_761,bert_dim_762,bert_dim_763,bert_dim_764,bert_dim_765,bert_dim_766,bert_dim_767,target
0,-0.045296,0.244643,0.146881,-0.101254,-0.354191,-0.580979,1.203313,0.624728,0.039920,-0.574070,...,-0.052892,0.023292,-0.154176,0.340931,-0.142372,-0.014048,-0.112122,0.264723,-0.180618,1
1,-0.833856,-0.186808,-0.461408,-0.482215,0.191647,0.173931,0.226668,0.462386,-0.625819,-0.180016,...,-0.069476,-0.059576,-0.754987,0.476386,-0.163974,-0.251988,-0.121008,0.281774,-0.028574,1
2,-0.553840,-0.868089,0.456105,-0.360056,0.596364,-0.175732,0.661811,0.303952,-0.157333,-0.344494,...,-0.411147,0.700571,-0.238418,-0.197669,-0.201720,-0.384008,0.014200,0.099776,0.380442,1
3,-0.709926,-0.059643,-0.065534,-0.215128,-0.341166,-0.002201,0.697558,0.685646,-0.670927,-0.089965,...,0.297650,0.686873,-0.404352,0.475008,0.230612,-0.003604,-0.798723,0.419383,0.010650,1
4,-0.127495,-0.026461,0.160941,0.114806,-0.279382,-0.752050,1.035819,0.550378,0.148431,-0.372095,...,0.057954,-0.508310,-0.334053,0.124174,0.029579,-0.142547,-0.883545,0.055623,0.213822,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.013373,0.053648,-0.172382,-0.199139,-0.172104,0.180905,0.537056,0.874261,-0.208165,-0.178274,...,-0.085405,-0.255601,-0.167974,0.482883,0.600995,-0.127468,-0.105035,0.021592,0.195847,1
7609,0.113711,0.180877,-0.105192,-0.032074,-0.497120,-0.442231,0.585394,0.361678,0.157206,-0.619585,...,-0.099414,-0.326833,-0.415066,0.137362,0.064457,-0.309373,-0.617580,0.308346,0.432512,1
7610,-0.218554,0.051987,0.221798,-0.167987,-0.171388,-0.090077,0.484971,0.561129,-0.152184,-0.275283,...,0.069041,-0.325151,-0.518002,0.083724,0.004744,0.034347,-0.159692,0.606469,0.353067,1
7611,-0.439928,-0.425846,0.083126,0.157701,-0.386743,0.026565,0.299643,0.243440,-0.287628,-0.245310,...,-0.009644,0.338429,0.078797,0.039647,-0.666714,0.179503,0.136507,0.158583,0.207595,1


In [2]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np

X = train.drop(columns = ['target'])
y = train['target']

# Сделайте разделение даты на трейн и валидацию с помощью train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Эту ячейку не изменяем до определенного этапа

class Head_catboost():
    def __init__(self,):
        # это основные параметры катбуста, которые пока что не изменяйте
        self.params = {
            'iterations': 1000,
            'learning_rate': 0.05,
            'depth': 6,
            'loss_function': 'Logloss',  # У нас бинарка
            'eval_metric': 'F1',  # Оптимизируем F1 как основной скор соревы
            'early_stopping_rounds': 150, 
            'random_seed': 42, # всегда фиксируйте сиды!
            'verbose': 100,
            'task_type': 'GPU' if torch.cuda.is_available() else 'CPU'
        } 
    
    def train(self, X_train, X_val, y_train, y_val):
        model = CatBoostClassifier(**self.params)
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            #plot=True, на кагле не заработает, но на личных машинках должен
            use_best_model=True # берется лучшая по скору на валидации
        )
        
        return model

Теперь перейдём к получению сабмита. Условием соревнования является предсказание только классов — 0 или 1. Однако CatBoost по умолчанию возвращает вероятности.

Я предложу вам два варианта формирования сабмита. Подумайте (или проверьте на практике), какой из них окажется лучше, и попробуйте объяснить, почему один метод работает лучше другого.

In [4]:
def predict_argmax(test):
    probs = model.predict_proba(test)
    predictions = np.argmax(probs, axis=1)
    
    sample['target'] = predictions
    sample.to_csv('Submission_arg.csv', index=False) 

In [21]:
def predict_argmax(test, model):
    probs = model.predict_proba(test)[:, 1]
    threshold = 0.65 # не изменяйте трешхолд до определенного этапа
    predictions = (probs >= threshold).astype(int)
    
    sample['target'] = predictions
    sample.to_csv('Submission_thr.csv', index=False)

(сабмиты можно найти во вкладке Output справа в меню кагла, после чего загрузить в соревнование)

Пишите ваш ответ тут: 
В случае первого метода, выбираются классы с наибольшей вероятностью. Первый метод использует жесткое правило argmax (порог 0.5), что не всегда оптимально.
Второй метод работает с вероятностями, что дает больше информации, чем просто выбор класса с максимальной вероятностью.

# Теперь будем создавать эмбеддинги самостоятельно

Вам предстоит попробовать несколько подходов к построению текстовых представлений.

Для каждого текста в трейне создайте эмбеддинг, после чего обучите CatBoost и посмотрите  полученный скор в соревновании. Мы посмотрим, какой метод покажет себя лучше всего.

In [6]:

import pandas as pd
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv').drop(columns=['keyword', 'location'])
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv').drop(columns=['keyword', 'location'])
sample = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
train

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...
7608,10869,Two giant cranes holding a bridge collapse int...,1
7609,10870,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,Police investigating after an e-bike collided ...,1


## 1. Bag-of-Words (BOW) (0.75 балла)

Метод создаёт числовое представление текста, где каждое слово превращается в отдельную колонку, а значение указывает частоту его встречаемости. Такой подход полностью игнорирует порядок слов и их смысловые связи, опираясь только на факт их наличия в тексте.

In [7]:
import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, accuracy_score, f1_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train['cleaned_text'] = train['text'].apply(preprocess_text)

In [8]:
vectorizer = CountVectorizer(
    max_features=5000,          # ограничиваем количество признаков
    stop_words='english',       # используем английские стоп-слова
    min_df=2,                   # игнорировать слова, встречающиеся менее 2 раз
    max_df=0.95,                # игнорировать слова, встречающиеся в >95% документов
    ngram_range=(1, 2)          # использовать униграммы и биграммы
)

bow_matrix_train = vectorizer.fit_transform(train['cleaned_text'])

bow_train = pd.DataFrame(bow_matrix_train.toarray(), columns=vectorizer.get_feature_names_out())
features = vectorizer.get_feature_names_out()
bow_train["other_words"] = np.zeros(len(bow_train))

In [9]:
X = bow_train
y = train['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [46]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

6090 1523 6090 1523


In [48]:
model = Head_catboost()
model = model.train(X_train, X_test, y_train, y_test)

0:	learn: 0.1330487	test: 0.1495063	best: 0.1495063 (0)	total: 19.1ms	remaining: 19.1s
100:	learn: 0.5509877	test: 0.5809129	best: 0.5809129 (100)	total: 1.62s	remaining: 14.5s
200:	learn: 0.6645692	test: 0.6691729	best: 0.6691729 (200)	total: 3.22s	remaining: 12.8s
300:	learn: 0.7287823	test: 0.7079964	best: 0.7099099 (291)	total: 4.79s	remaining: 11.1s
400:	learn: 0.7622472	test: 0.7277533	best: 0.7299912 (399)	total: 6.38s	remaining: 9.53s
500:	learn: 0.7873283	test: 0.7408696	best: 0.7426087 (476)	total: 7.98s	remaining: 7.95s
600:	learn: 0.8035910	test: 0.7484979	best: 0.7521515 (571)	total: 9.66s	remaining: 6.42s
700:	learn: 0.8143138	test: 0.7510693	best: 0.7521515 (571)	total: 11.3s	remaining: 4.81s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.752151463
bestIteration = 571

Shrink model to first 572 iterations.


In [49]:
X_matrix_test = vectorizer.fit_transform(test["text"])
bow_test = pd.DataFrame(X_matrix_test.toarray(), columns=vectorizer.get_feature_names_out())
bow_test["other_words"] = np.zeros(len(bow_test))

In [50]:
for i in bow_test.columns:
    if i not in features:
        bow_test["other_words"] += bow_test[i]
        bow_test.drop(i, axis=1, inplace=True)

In [51]:
for i in features:
    if i not in bow_test:
        bow_test[i] = pd.Series([0]*len(bow_test))

/tmp/ipykernel_36/1925401350.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bow_test[i] = pd.Series([0]*len(bow_test))
/tmp/ipykernel_36/1925401350.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bow_test[i] = pd.Series([0]*len(bow_test))
/tmp/ipykernel_36/1925401350.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

In [56]:
len(bow_test.columns)

5000

In [53]:
len(features)

5000

In [57]:
predict_argmax(bow_test)

**Полученный скор:** 0.74

## 2. TF-IDF (0.75 балла)

Учитывает не только частоту слова в документе (TF), но и его редкость во всей коллекции документов (IDF), автоматически понижая вес распространенных слов. Это позволяет выделять действительно значимые термины, которые характерны для конкретного текста.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

In [58]:
tfidf_matrix_train = vectorizer.fit_transform(train["text"])

In [59]:
tfidf_features = vectorizer.get_feature_names_out()

In [60]:
tfidf_train = pd.DataFrame(tfidf_matrix_train.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_train["other_words"] = np.zeros(len(tfidf_train))
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_train, y, test_size=0.2, train_size=0.8)
print(len(X_train_tfidf), len(X_test_tfidf), len(y_train_tfidf), len(y_test_tfidf))

6090 1523 6090 1523


In [62]:
model = Head_catboost()
model = model.train(X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf)

0:	learn: 0.5970422	test: 0.6319444	best: 0.6319444 (0)	total: 17.9ms	remaining: 17.9s
100:	learn: 0.6629296	test: 0.6463527	best: 0.6637103 (42)	total: 1.61s	remaining: 14.4s
200:	learn: 0.7059929	test: 0.6714542	best: 0.6726457 (198)	total: 3.18s	remaining: 12.6s
300:	learn: 0.7527372	test: 0.7060870	best: 0.7060870 (300)	total: 4.76s	remaining: 11.1s
400:	learn: 0.7805747	test: 0.7172414	best: 0.7183463 (376)	total: 6.33s	remaining: 9.45s
500:	learn: 0.8017563	test: 0.7285106	best: 0.7285106 (497)	total: 7.91s	remaining: 7.88s
600:	learn: 0.8179443	test: 0.7390572	best: 0.7390572 (600)	total: 9.48s	remaining: 6.29s
700:	learn: 0.8290358	test: 0.7372811	best: 0.7401838 (675)	total: 11.1s	remaining: 4.73s
800:	learn: 0.8365364	test: 0.7441860	best: 0.7441860 (727)	total: 12.7s	remaining: 3.15s
900:	learn: 0.8464017	test: 0.7462687	best: 0.7477254 (835)	total: 14.3s	remaining: 1.57s
999:	learn: 0.8513282	test: 0.7506173	best: 0.7514451 (941)	total: 15.8s	remaining: 0us

bestTest = 0.75

In [65]:
tfidf_matrix_test = vectorizer.fit_transform(test["text"])
tfidf_test = pd.DataFrame(tfidf_matrix_test.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_test["other_words"] = np.zeros(len(tfidf_test))

In [66]:
for i in tfidf_test.columns:
    if i not in tfidf_features:
        tfidf_test["other_words"] += tfidf_test[i]
        tfidf_test.drop(i, axis=1, inplace=True)

In [67]:
for i in tfidf_features:
    if i not in tfidf_test:
        tfidf_test[i] = np.zeros(len(tfidf_test))
        
print(len(tfidf_features), len(tfidf_test.columns))

/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_test[i] = np.zeros(len(tfidf_test))
/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_test[i] = np.zeros(len(tfidf_test))
/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `n

5000 5000


/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_test[i] = np.zeros(len(tfidf_test))
/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tfidf_test[i] = np.zeros(len(tfidf_test))
/tmp/ipykernel_36/2164882787.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `n

In [68]:
predict_argmax(tfidf_test)

**Полученный скор:** 0.77

## 3. Word2Vec (0.75 балла)

Нейронная сеть преобразует слова в векторы фиксированной длины так, что семантически близкие слова оказываются рядом в пространстве. Обучение идёт либо в режиме skip-gram (слово → контекст), либо CBOW (контекст → слово), что позволяет сохранять смысловые связи.

In [69]:
from gensim.models import Word2Vec

tokenized_texts = train["text"].apply(lambda x: x.split()).tolist()

w2v_model = Word2Vec(
    sentences=tokenized_texts,  
    vector_size=256,
    window=9,
    min_count=1,
    workers=4,
    epochs=10 
)

In [ ]:
tokenized_texts = train["text"].apply(lambda x: x.split()).tolist()

In [73]:
def get_document_embedding(tokens, w2v_model):
    word_vectors = []
    for token in tokens:
        if token in w2v_model.wv:
            word_vectors.append(w2v_model.wv[token])
    
    if not word_vectors:
        return np.zeros(w2v_model.vector_size)
    
    return np.mean(word_vectors, axis=0)
   
X_w2v = np.array([get_document_embedding(doc, w2v_model) for doc in tokenized_texts])

In [74]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, train_size=0.8)

In [76]:
model = Head_catboost()
model = model.train(X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v)

0:	learn: 0.6158445	test: 0.5764796	best: 0.5764796 (0)	total: 59.5ms	remaining: 59.5s
100:	learn: 0.6502298	test: 0.5837563	best: 0.5875519 (26)	total: 3.62s	remaining: 32.2s
200:	learn: 0.6820589	test: 0.5863248	best: 0.5937235 (136)	total: 7.18s	remaining: 28.5s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.5937234945
bestIteration = 136

Shrink model to first 137 iterations.


In [78]:
tokenized_texts_test = test["text"].apply(lambda x: x.split()).tolist()

In [80]:
test_w2v = np.array([get_document_embedding(doc, w2v_model) for doc in tokenized_texts_test])

In [83]:
predict_argmax(test_w2v)

**Полученный скор:** 0.65

## 4. GloVe (0.75 балла)

Строит векторные представления слов на основе статистики их совместной встречаемости во всем корпусе текстов, вычисляя, насколько часто пары слов появляются вместе. Оптимизирует вектора так, чтобы их скалярное произведение соответствовало логарифму вероятности совместного появления слов.

In [9]:
from gensim.scripts.glove2word2vec import glove2word2vec
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!cd glove.6B/

--2025-09-21 14:43:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-09-21 14:43:25--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-09-21 14:43:25--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [10]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'

glove2word2vec(glove_input_file, word2vec_output_file)

/tmp/ipykernel_36/1040684936.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 100)

In [11]:
from gensim.models import KeyedVectors
import re

model = KeyedVectors.load_word2vec_format(word2vec_output_file)  # загрузка конвертированной модели

In [12]:
#train_for_glove = train["text"].apply(train['cleaned_text']['text'])  
train_for_glove = train["cleaned_text"].apply(lambda x: x.split())
train_for_glove

0       [our, deeds, are, the, reason, of, this, may, ...
1           [forest, fire, near, la, ronge, sask, canada]
2       [all, residents, asked, to, shelter, in, place...
3       [people, receive, evacuation, orders, in, cali...
4       [just, got, sent, this, photo, from, ruby, as,...
                              ...                        
7608    [two, giant, cranes, holding, a, bridge, colla...
7609    [the, out, of, control, wild, fires, in, calif...
7610                   [m, utckm, s, of, volcano, hawaii]
7611    [police, investigating, after, an, ebike, coll...
7612    [the, latest, more, homes, razed, by, northern...
Name: cleaned_text, Length: 7613, dtype: object

In [13]:
def text_to_vector(tokens, model, vector_size=100):
    """Преобразование списка слов в вектор документа"""
    vectors = []
    for word in tokens:
        if word in model:  # если слово есть в словаре модели
            vectors.append(model[word])  # добавляем его вектор
    
    if len(vectors) == 0:  # если ни одного слова не найдено в словаре
        return np.zeros(vector_size)  # возвращаем нулевой вектор
    
    return np.mean(vectors, axis=0)  # возвращаем среднее векторов слов

# Применяем функцию к каждому документу
train_glove = train_for_glove.apply(lambda x: text_to_vector(x, model))
train_glove  # Series с векторами документов

0       [0.024025166, 0.30114317, 0.5709731, -0.348697...
1       [-0.09041647, -0.33786145, 0.27815086, 0.42031...
2       [-0.0023915009, 0.19133674, 0.12953609, -0.262...
3       [0.24563384, 0.20827217, 0.06651084, -0.110012...
4       [0.005269987, 0.2079581, 0.26436365, -0.418603...
                              ...                        
7608    [0.077938, 0.3160525, -0.07051642, -0.02852789...
7609    [-0.17132111, -0.0532791, 0.59264404, -0.16314...
7610    [-0.099154, 0.24698201, 0.52519196, 0.17278, -...
7611    [0.03451259, -0.13638633, 0.15891287, -0.11515...
7612    [-0.15976849, -0.008933826, 0.29539907, -0.231...
Name: cleaned_text, Length: 7613, dtype: object

In [27]:
X_glove = np.vstack(train_glove.values)

In [28]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(
    X_glove, y, test_size=0.2, random_state=42
)

In [15]:
#не нужно
X_train_glove = X_train_glove.reset_index(drop=True)  
X_test_glove = X_test_glove.reset_index(drop=True)
y_train_glove = y_train_glove.reset_index(drop=True)
y_test_glove = y_test_glove.reset_index(drop=True)

In [29]:
model_cat = Head_catboost()
model_cat = model_cat.train(X_train_glove, X_test_glove, y_train_glove, y_test_glove)

0:	learn: 0.6580508	test: 0.6620330	best: 0.6620330 (0)	total: 52.9ms	remaining: 52.8s
100:	learn: 0.7818709	test: 0.7468672	best: 0.7483333 (95)	total: 2.97s	remaining: 26.4s
200:	learn: 0.8177340	test: 0.7625418	best: 0.7646077 (188)	total: 5.75s	remaining: 22.8s
300:	learn: 0.8371714	test: 0.7661692	best: 0.7661692 (299)	total: 8.54s	remaining: 19.8s
400:	learn: 0.8547566	test: 0.7642680	best: 0.7682119 (302)	total: 11.3s	remaining: 16.8s
500:	learn: 0.8689823	test: 0.7650453	best: 0.7702479 (432)	total: 14s	remaining: 13.9s
bestTest = 0.7702479339
bestIteration = 432
Shrink model to first 433 iterations.


In [30]:
test['cleaned_text'] = test['text'].apply(preprocess_text)

In [31]:
test_tokens = test['cleaned_text'].apply(lambda x: x.split())

In [32]:
test_glove_vectors = test_tokens.apply(lambda x: text_to_vector(x, model))

In [33]:
test_glove_array = np.vstack(test_glove_vectors.values)

In [34]:
predictions = predict_argmax(test_glove_array, model_cat)

**Полученный скор:** 0.78

## 5. Трансформерный подход (0.75 балла)

Для работы с NLP моделями используйте сайт https://huggingface.co/ и их библиотеку transformers. Сначала будем использовать базовую версию BERT https://huggingface.co/google-bert/bert-base-uncased 

По желанию можете еще сделать вариант с использованием Pipeline

In [10]:
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset, DataLoader
import re
from transformers import AutoTokenizer, AutoModel, get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.optim as optim
import numpy as np
from tqdm import tqdm

2025-09-16 15:05:06.689995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758035107.006583      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758035107.104423      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Bert(nn.Module):
    def __init__(self, trainable=False, num_classes=1):
        super().__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.bert = AutoModel.from_pretrained('bert-base-uncased').to(self.device) 
        self.CLS = 0
        
        self.classifier = nn.Linear(768, num_classes).to(self.device)

        if not trainable:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_texts):
        tokens = self.tokenizer(
            input_texts, 
            padding=True, 
            truncation=True, 
            max_length=512, 
            return_tensors='pt'
        ).to(self.device)
        if not any(param.requires_grad for param in self.bert.parameters()):
            with torch.no_grad():
                output = self.bert(**tokens)
        else:
            output = self.bert(**tokens)
        cls_embeddings = output.last_hidden_state[:, 0, :]
        
        logits = self.classifier(cls_embeddings)
        return logits.squeeze()

In [ ]:
class DatasetBuilder(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], torch.tensor(self.y[idx], dtype=torch.float)

In [ ]:
class ModelUsage:
    def __init__(self, model):
        self.model = model
        self.model.to(device)

    def train(self, dataloader, optimizer, loss_func, scheduler, epochs, val_dataloader):
        self.model.train()
        for epoch in range(epochs):
            total_loss = 0
            for texts, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}", colour="GREEN"):
                labels = labels.to(device)
                optimizer.zero_grad()
                output = self.model(texts) 
                loss = loss_func(output, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(dataloader)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}")
            self.test(val_dataloader, loss_func)
    def test(self, dataloader, loss_func):
        self.model.eval()
        testloss, all_labels, all_preds = 0, [], []
        num_batches = len(dataloader)

        with torch.no_grad():
            for texts, labels in tqdm(dataloader, desc="Eval", colour="CYAN"):
                labels = labels.to(device)
                output = self.model(texts)
                loss = loss_func(output, labels)
                testloss += loss.item()

                probs = torch.sigmoid(output).cpu().numpy()
                all_preds.append(probs)
                all_labels.append(labels.cpu().numpy())
        
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        predictions = (all_preds >= 0.5).astype(int)
        accuracy = accuracy_score(all_labels, predictions)
        
        testloss /= num_batches
        print(f"Test Loss: {testloss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("-" * 50)



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train['text'].values.tolist(), 
    train['target'].values.tolist(), 
    test_size=0.2,
    random_state=42
)

train_dataset = DatasetBuilder(X_train, y_train)
val_dataset = DatasetBuilder(X_val, y_val)

batch_train_size = 16
batch_val_size = 16

train_dl = DataLoader(
    train_dataset,
    batch_size=batch_train_size,
    num_workers=2,
    shuffle=True,
)

val_dl = DataLoader(
    val_dataset,
    batch_size=batch_val_size,
    num_workers=2,
    shuffle=False,
)

In [ ]:
model = Bert(trainable=True, num_classes=1)

In [ ]:
epochs = 3
LR = 2e-5
optimizer = optim.AdamW([
    {'params': model.bert.parameters(), 'lr': 2e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
], weight_decay=0.01)

loss_func = nn.BCEWithLogitsLoss()

total_steps = len(train_dl) * epochs
scheduler = get_cosine_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=int(0.1 * total_steps), 
    num_training_steps=total_steps
)

In [ ]:
model_f = ModelUsage(model)

In [85]:
model_f.train(train_dl, optimizer, loss_func, scheduler, epochs, val_dl)

Epoch 1:   0%|          | 0/381 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 1: 100%|██████████| 381/381 [31:06<00:00,  4.90s/it]


Epoch 1/3, Train Loss: 0.4601


Eval:   0%|          | 0/96 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Eval: 100%|██████████| 96/96 [02:13<00:00,  1.39s/it]


Test Loss: 0.3773
Accuracy: 0.8483
--------------------------------------------------


Epoch 2:   0%|          | 0/381 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 2: 100%|██████████| 381/381 [29:25<00:00,  4.63s/it]


...

Test Loss: 0.3983
Accuracy: 0.8385
--------------------------------------------------


Epoch 3:   0%|          | 0/381 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 3: 100%|██████████| 381/381 [29:09<00:00,  4.59s/it]


Epoch 3/3, Train Loss: 0.1537


Eval:   0%|          | 0/96 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Eval: 100%|██████████| 96/96 [02:23<00:00,  1.50s/it]

Test Loss: 0.4804
Accuracy: 0.8365
--------------------------------------------------


In [ ]:
def predict_test_data(model, test_texts, batch_size=16):
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(test_texts), batch_size), desc="Predicting test data"):
            batch_texts = test_texts[i:i + batch_size]
            output = model(batch_texts)
            probs = torch.sigmoid(output).cpu().numpy()
            all_probs.append(probs)
    
    all_probs = np.concatenate(all_probs)
    predictions = (all_probs >= 0.5).astype(int)
    
    return all_probs, predictions


def create_submission(test_data, predictions, output_file='submission.csv'):
    submission = pd.DataFrame({
        'id': test_data['id'],
        'target': predictions
    })
    
    submission.to_csv(output_file, index=False)
    print(f"Submission файл сохранен как {output_file}")
    return submission


test_probs, test_preds = predict_test_data(model, test['text'].tolist())
submission = create_submission(test, test_preds, 'bert_submission.csv')

print(f"Распределение предсказаний: {np.bincount(test_preds)}")
print(f"Средняя вероятность: {test_probs.mean():.4f}")

Predicting test data: 100%|██████████| 204/204 [04:46<00:00,  1.40s/it]

Submission файл сохранен как bert_submission.csv
Распределение предсказаний: [1975 1288]
Средняя вероятность: 0.4230


Скор: 0.83

## 6. Fine-Tuning (2.75 баллов)

Теперь сделаем файнтюнинг BERT для нашей задачи, добавим линейный слой на CLS токен и дообучим полученную модель на наши данные. Файнтюнинг самый эффективный метод, если у вас есть ресурсы и время для обучения.

Как я показывал на семенаре можно обучать с помощью torch или с помощью Trainer из библиотеки transformers. 
Реализуйте оба варианта. Но в коротких соревнованиях отдавайте предпочтение Trainer

В нашем случае мы можем воспринимать задачу как регрессию так и классификацию, это будет зависить от выбранного вами подхода обучения линейного слоя на CLS токене - линейный слой размерностью (d_cls_Berta, 1) - регрессия или же (d_cls_Berta, 2) - классификация. Лично я использовал бы классификацию и обучал бы на CrossEntropyLoss. Пока что реализуйте эту идею, потом можете поэксперементировать.

### HF Trainer

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"

# os.environ["WANDB_MODE"] = "disabled"


from transformers import (
    BertTokenizer, 
    BertForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset

from sklearn.metrics import f1_score


model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,  # Количество классов
)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
X_train = train['text']
y_train = train['target']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

train_ds = Dataset.from_dict({
    'text': X_train,
    'label': y_train
})
val_ds = Dataset.from_dict({
    'text': X_val,
    'label': y_val
})

def tokenize_fn(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=128)

train_ds = train_ds.map(tokenize_fn, batched=True)
val_ds = val_ds.map(tokenize_fn, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"f1": f1_score(labels, preds)}



In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [57]:
# используйте эти параметры обучения
training_args = TrainingArguments(
    # Основные параметры
    output_dir='./bert-binary-classifier',  # Директория для сохранения
    
    # Параметры обучения
    num_train_epochs=3,                     # Количество эпох
    per_device_train_batch_size=32,         # Размер батча для обучения
    per_device_eval_batch_size=32,          # Размер батча для валидации
    learning_rate=2e-5,                     # Learning rate
    warmup_ratio = 0.1,                      # 10% от общего числа шагов для вармапа или warmup_steps = int(0.1 * total_training_steps)
    lr_scheduler_type = 'cosine',            # Можете посмотреть на них в 
                                            # https://www.kaggle.com/code/snnclsr/learning-rate-schedulers 
                                            # соответсвующий ему будет get_cosine_schedule_with_warmup
    
    # Сохранение и логирование
    logging_dir='./logs',                   # Директория для логов
    logging_steps=100,                      # Частота логирования
    save_steps=500,                         # Частота сохранения
    save_total_limit=2,                     # Максимум чекпоинтов
    save_strategy='epoch',                  # Стратегия сохранения
    
    # Валидация
    eval_strategy='epoch',            # Стратегия валидации
    eval_steps=500,                         # Шаги для валидации
    load_best_model_at_end=True,            # Загружать лучшую модель
    metric_for_best_model='f1',             # Метрика для выбора лучшей
    greater_is_better=True,                 # Больше значение = лучше

    # воспроизводимость
    seed=42,                                # Seed для воспроизводимости
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # report_to="none"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_36/3206430980.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [58]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.379214,0.814274
2,0.505800,0.381490,0.811437
3,0.354800,0.405033,0.812065


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=288, training_loss=0.38984153005811906, metrics={'train_runtime': 190.5113, 'train_samples_per_second': 95.9, 'train_steps_per_second': 1.512, 'total_flos': 788634277339200.0, 'train_loss': 0.38984153005811906, 'epoch': 3.0})

In [60]:
test_ds = Dataset.from_dict({
    'text': test['text'] 
})

test_ds = test_ds.map(tokenize_fn, batched=True)
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask'])


predictions = trainer.predict(test_ds)
predicted_probs = torch.softmax(torch.tensor(predictions.predictions), dim=-1)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# 4. Создание submission файла
# Предполагается, что sample DataFrame существует с колонкой 'target'
sample['target'] = predicted_labels
sample.to_csv('bert_submission.csv', index=False)

print("Submission файл создан: bert_submission.csv")

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

Submission файл создан: bert_submission.csv


Скор: 0.83266

### Torch

In [95]:
from sklearn.metrics import roc_auc_score
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_df = train_data[['text', 'target']]
#train_df = train[['text', 'target']]
train_split, val_split = train_test_split(train_df, test_size=0.2, random_state=42)

In [85]:
import torch
import torch.nn as nn

In [86]:
class Bert(nn.Module):
    def __init__(self, trainable=False, num_labels=2):
        super().__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.bert = AutoModel.from_pretrained('bert-base-uncased').to(self.device) 
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, num_labels).to(self.device)
        if not trainable:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_texts):
        """
        Получаем текстовый вход, токенизирует его и пропускает через BERT-модель,
        возвращая предсказания для multi-label classification.
        """
        # Токенизация
        tokens = self.tokenizer(
            input_texts, 
            padding=True, 
            truncation=True, 
            max_length=256, 
            return_tensors='pt'
        ).to(self.device)
        
        # Пропускаем через BERT
        output = self.bert(**tokens)
        
        # Берем [CLS] токен
        cls_embeddings = output.last_hidden_state[:, 0, :]
        
        # Пропускаем через классификатор
        logits = self.classifier(cls_embeddings)
        
        return logits

In [87]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor(self.labels[idx], dtype=torch.long)


In [105]:
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Убедимся, что idx - это одиночное значение, а не список
        if isinstance(idx, list):
            # Если передали список индексов, берем первый (для совместимости)
            idx = idx[0]
        return self.texts[idx], torch.tensor(self.labels[idx], dtype=torch.long)

    def __getitems__(self, keys):
        """Метод для обработки батчей индексов"""
        return [self.__getitem__(key) for key in keys]

In [88]:
def compute_metrics(all_preds, all_labels):
    probs = torch.softmax(torch.tensor(all_preds), dim=1).numpy()
    labels = np.array(all_labels)
    
    try:
        auc = roc_auc_score(labels, probs[:, 1])  # Вероятность класса 1
        return {"auc": auc}
    except ValueError:
        return {"auc": 0}

In [89]:
def train_model(model, train_loader, val_loader, epochs=2, lr=2e-5):
    device = model.device
    
    optimizer = optim.AdamW([
        {'params': model.bert.parameters(), 'lr': 2e-5},
        {'params': model.classifier.parameters(), 'lr': 1e-3}
    ], weight_decay=0.01)
    
    # Scheduler
    total_steps = len(train_loader) * epochs
    scheduler = get_cosine_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=int(0.1 * total_steps), 
        num_training_steps=total_steps
    )
    
    loss_func = nn.CrossEntropyLoss()
    
    best_auc = 0
    for epoch in range(epochs):
        # Training
        model.train()
        total_loss = 0
        
        for texts, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            labels = labels.to(device)
            optimizer.zero_grad()
            
            output = model(texts)
            loss = loss_func(output, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for texts, labels in val_loader:
                labels = labels.to(device)
                output = model(texts)
                loss = loss_func(output, labels)
                val_loss += loss.item()
                
                all_preds.append(output.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
        
        # Вычисление метрик
        all_preds = np.vstack(all_preds)
        all_labels = np.concatenate(all_labels)  
        metrics = compute_metrics(all_preds, all_labels)
        
        print(f"Epoch {epoch+1}:")
        print(f"Train Loss: {total_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("-" * 50)
        
        # Сохраняем лучшую модель
        if metrics['auc'] > best_auc:
            best_auc = metrics['auc']
            torch.save(model.state_dict(), 'best_model.pth')
    
    return model

In [90]:
def predict_test_data(model, test_texts, batch_size=16):
    """Предсказание на test данных"""
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(test_texts), batch_size), desc="Predicting"):
            batch_texts = test_texts[i:i + batch_size]
            output = model(batch_texts)
            # Для бинарной классификации берем вероятность класса 1
            probs = torch.softmax(output, dim=1)[:, 1].cpu().numpy()
            all_probs.append(probs)
    
    return np.concatenate(all_probs)

In [91]:
def create_submission(test_df, probs, output_file='submission.csv'):
    """Создание submission файла"""
    # ОШИБКА: для бинарной классификации только один столбец 'target'
    submission = pd.DataFrame({
        "id": test_df["id"],
        "target": probs
    })
    submission.to_csv(output_file, index=False)
    print(f"Submission файл сохранен: {output_file}")
    return submission


In [96]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [117]:
def predict_test_data(model, test_texts, batch_size=16):
    """Предсказание на test данных - возвращаем классы, а не вероятности"""
    model.eval()
    all_predictions = []  # Изменено: сохраняем предсказанные классы
    
    with torch.no_grad():
        for i in tqdm(range(0, len(test_texts), batch_size), desc="Predicting"):
            batch_texts = test_texts[i:i + batch_size]
            output = model(batch_texts)
            # Берем argmax для получения предсказанных классов (0 или 1)
            predictions = torch.argmax(output, dim=1).cpu().numpy()
            all_predictions.append(predictions)
    
    return np.concatenate(all_predictions)  # Возвращаем массив классов
def create_submission(test_df, predictions, output_file='submission.csv'):
    """Создание submission файла с классами"""
    submission = pd.DataFrame({
        "id": test_df["id"],
        "target": predictions.astype(int)  # Преобразуем в целые числа
    })
    submission.to_csv(output_file, index=False)
    print(f"Submission файл сохранен: {output_file}")
    print(f"Распределение предсказаний: {pd.Series(predictions).value_counts().to_dict()}")
    return submission

In [118]:
def train():
    """Основная функция обучения"""
    # Предполагается, что train_ds, val_ds, test_ds, test_df определены глобально
    # Подготовка данных
    train_texts = train_split['text'].tolist()
    train_labels = train_split['target'].tolist()
    val_texts = val_split['text'].tolist()
    val_labels = val_split['target'].tolist()
    test_texts = test_data['text'].tolist()
    
    # Создание DataLoader
    train_dataset = ToxicCommentsDataset(train_texts, train_labels)
    val_dataset = ToxicCommentsDataset(val_texts, val_labels)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    
    # Инициализация модели - для бинарной классификации num_labels=2
    model = Bert(trainable=True, num_labels=2)
    # Обучение
    print("Начинаем обучение BERT...")
    model = train_model(model, train_loader, val_loader, epochs=2)
    
    # Предсказание на test данных
    print("Делаем предсказания на test данных...")
    test_probs = predict_test_data(model, test_texts)
    
    # Создание submission
    submission = create_submission(test_data, test_probs)
    
    print("Обучение завершено!")
    return model, submission

In [103]:
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
from transformers import get_cosine_schedule_with_warmup
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
if __name__ == "__main__":
    model, submission = train()

Начинаем обучение BERT...


Epoch 1/2:  25%|██▍       | 94/381 [00:16<00:49,  5.75it/s]

In [ ]:
submission = pd.read_csv('/kaggle/working/submission.csv')

In [ ]:
submission

Скор: 0.83328

# Для самых пытливых, которые хотят участвовать и выигрывать в соревнованиях (3 балла)

Теперь я даю вам полную свободу в эксперементах, записывайте логи и идеи ниже, попробуйте занять хорошее место в этой соревке. 

Первым делом я бы предложил поперебирать модели. Для удобства в подборе можно использовать AutoModelForSequenceClassification. 

Для выбора моделей воспользуйтесь рейтингом MTEB - https://huggingface.co/spaces/mteb/leaderboard (новая версия) / https://huggingface.co/spaces/mteb/leaderboard_legacy (легаси версия)

Можете еще посмотреть библиотеку SentenceTransformer (является оберткой для Transformers от сбера), но я ей не то чтобы много пользовался, также у нее куча проблем в плане совместимостей и на всеросе она мне куду положила)))

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (roc_auc_score, f1_score, precision_score, 
                            recall_score, accuracy_score, confusion_matrix)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from transformers import (AutoTokenizer, AutoModel, 
                         get_cosine_schedule_with_warmup)
import warnings
warnings.filterwarnings('ignore')
import optuna
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
class Config:
    seed = 42
    test_size = 0.2
    n_folds = 5
    batch_size = 16
    max_length = 256
    n_trials = 20
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
def advanced_features(df, text_column):
    df = df.copy()
    
    # Базовые текстовые статистики
    df['text_length'] = df[text_column].str.len()
    df['word_count'] = df[text_column].str.split().str.len()
    df['char_count'] = df[text_column].str.replace(r'\s', '', regex=True).str.len()
    df['avg_word_length'] = df['char_count'] / df['word_count'].replace(0, 1)
    
    # Пунктуация и символы
    df['exclamation_count'] = df[text_column].str.count('!')
    df['question_count'] = df[text_column].str.count('\?')
    df['capital_ratio'] = df[text_column].apply(
        lambda x: sum(1 for c in x if c.isupper()) / max(len(x), 1)
    )
    df['digit_count'] = df[text_column].str.count(r'\d')
    df['special_char_count'] = df[text_column].str.count(r'[^\w\s]')
    
    # Семантические фичи
    df['has_http'] = df[text_column].str.contains('http').astype(int)
    df['has_www'] = df[text_column].str.contains('www').astype(int)
    df['has_mention'] = df[text_column].str.contains('@').astype(int)
    df['has_hashtag'] = df[text_column].str.contains('#').astype(int)
    
    # Стилистические фичи
    df['unique_words_ratio'] = df[text_column].apply(
        lambda x: len(set(x.split())) / max(len(x.split()), 1)
    )
    df['long_words_count'] = df[text_column].apply(
        lambda x: sum(1 for word in x.split() if len(word) > 6)
    )
    
    # Время суток (если есть timestamp)
    if 'timestamp' in df.columns:
        df['hour'] = pd.to_datetime(df['timestamp']).dt.hour
        df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
    
    return df

In [22]:
def run_feature_engineering_experiments(train_df, test_df, text_column='text', target_column='target'):
    results = [] #эксперименты с фичами в тексте
    
    # 1. Только текстовые фичи
    train_fe = advanced_features(train_df, text_column)
    test_fe = advanced_features(test_df, text_column)
    
    feature_columns = [col for col in train_fe.columns if col not in [text_column, target_column, 'timestamp']]
    
    X = train_fe[feature_columns]
    y = train_fe[target_column]
    
    model = XGBClassifier(random_state=Config.seed)
    cv_scores = cross_val_score(model, X, y, scoring='roc_auc', cv=3)
    score = cv_scores.mean()
    
    results.append({
        'experiment': 'only_engineered_features',
        'roc_auc': score,
        'features': feature_columns
    })
    
    # 2. TF-IDF + текстовые фичи
    vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
    X_tfidf = vectorizer.fit_transform(train_df[text_column]).toarray()
    X_combined = np.hstack([X_tfidf, X.values])
    
    model = XGBClassifier(random_state=Config.seed)
    cv_scores = cross_val_score(model, X_combined, y, scoring='roc_auc', cv=3)
    score = cv_scores.mean()
    
    results.append({
        'experiment': 'tfidf_plus_engineered',
        'roc_auc': score,
        'features': f'TF-IDF + {len(feature_columns)} features'
    })
    
    # 3. Только TF-IDF
    model = XGBClassifier(random_state=Config.seed)
    cv_scores = cross_val_score(model, X_tfidf, y, scoring='roc_auc', cv=3)
    score = cv_scores.mean()
    
    results.append({
        'experiment': 'only_tfidf',
        'roc_auc': score,
        'features': 'TF-IDF only'
    })
    
    return pd.DataFrame(results)

In [15]:
def run_model_experiments(X, y): #эксперименты с моделями
    results = []
    
    models = {
        'LogisticRegression': LogisticRegression(max_iter=1000, random_state=Config.seed),
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=Config.seed),
        'XGBoost': XGBClassifier(random_state=Config.seed),
        'LightGBM': LGBMClassifier(random_state=Config.seed),
        'CatBoost': CatBoostClassifier(verbose=0, random_state=Config.seed)
    }
    
    for name, model in models.items():
        print(f"модель {name}")
        cv_scores = cross_val_score(model, X, y, scoring='roc_auc', cv=3)
        results.append({
            'model': name,
            'roc_auc': cv_scores.mean(),
            'std': cv_scores.std(),
            'fit_time': measure_fit_time(model, X, y)
        })
    
    return pd.DataFrame(results)

In [16]:
def measure_fit_time(model, X, y): #измерение времени обучения
    import time
    start = time.time()
    model.fit(X, y)
    return time.time() - start

In [17]:
class EnhancedBertDataset(Dataset):
    def __init__(self, texts, labels, features=None):
        self.texts = texts
        self.labels = labels
        self.features = features  #дополнительные фичи
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        if self.features is not None:
            return self.texts[idx], self.features[idx], self.labels[idx]
        return self.texts[idx], self.labels[idx]


In [18]:
class EnhancedBert(nn.Module):
    def __init__(self, model_name='bert-base-uncased', num_features=0, dropout=0.3):
        super().__init__()
        self.device = Config.device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.bert = AutoModel.from_pretrained(model_name).to(self.device)
        self.dropout = nn.Dropout(dropout)
        
        # Комбинируем BERT embeddings с дополнительными фичами
        self.classifier = nn.Linear(768 + num_features, 1)
    
    def forward(self, texts, features=None):
        inputs = self.tokenizer(
            texts, return_tensors='pt', padding=True, 
            truncation=True, max_length=Config.max_length
        ).to(self.device)
        
        outputs = self.bert(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        # Комбинируем с дополнительными фичами
        if features is not None:
            features = features.float().to(self.device)
            combined = torch.cat([cls_embedding, features], dim=1)
        else:
            combined = cls_embedding
        
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        return logits.squeeze()

In [23]:
all_results = []

fe_results = run_feature_engineering_experiments(train, test, train['text'], train['target'])
all_results.extend(fe_results.to_dict('records'))

KeyError: "None of [Index(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',\n       'Forest fire near La Ronge Sask. Canada',\n       'All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected',\n       '13,000 people receive #wildfires evacuation orders in California ',\n       'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',\n       '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',\n       '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',\n       'I'm on top of the hill and I can see a fire in the woods...',\n       'There's an emergency evacuation happening now in the building across the street',\n       'I'm afraid that the tornado is coming to our area...',\n       ...\n       'Officials say a quarantine is in place at an Alabama

In [ ]:
train_fe = create_advanced_features(train, train['text'])
feature_columns = [col for col in train_fe.columns if col not in [text_column, target_column, 'timestamp']]

In [ ]:
X = train_fe[train['text']]
y = train_fe[train['target']]

In [ ]:
model_results = run_model_experiments(X, y)
all_results.extend(model_results.to_dict('records'))

In [ ]:
bert_results = run_bert_experiments(train, train['text'], train['target'])
all_results.extend(bert_results)

In [ ]:
ensemble_results = run_ensemble_experiments(X, y)
all_results.extend(ensemble_results)

In [ ]:
pd.DataFrame(all_results)

In [ ]:
#эксперименты с bert
X_train, X_val, y_train, y_val = train_test_split(
        train['text'], train['target'],
        test_size=Config.test_size, random_state=Config.seed, stratify=train['target'])

In [ ]:
train_dataset = TextDataset(X_train.tolist(), y_train.values)
val_dataset = TextDataset(X_val.tolist(), y_val.values)
    
train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=Config.batch_size)

In [ ]:
model = AdvancedBert()
model = train_bert(model, train_loader, val_loader)
score = evaluate_model(model, val_loader)

results.append({
        'experiment': 'bert_base',
        'roc_auc': score,
        'description': 'Базовая BERT модель'
    })

In [ ]:
#берт с фичами
train_fe = create_advanced_features(pd.DataFrame({train['text']: X_train, train['target']: y_train}), train['text'])
val_fe = create_advanced_features(pd.DataFrame({train['text']: X_val, train['target']: y_val}), train['text'])

In [ ]:
feature_columns = [col for col in train_fe.columns if col not in [train['text']:, train['target']:, 'timestamp']]

In [ ]:
train_dataset = EnhancedBertDataset(X_train.tolist(), y_train.values, train_fe[feature_columns].values)
val_dataset = EnhancedBertDataset(X_val.tolist(), y_val.values, val_fe[feature_columns].values)

train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=Config.batch_size)

In [ ]:
model = EnhancedBert(num_features=len(feature_columns))
model = train_enhanced_bert(model, train_loader, val_loader)
score = evaluate_enhanced_model(model, val_loader)

results.append({
        'experiment': 'bert_with_features',
        'roc_auc': score,
        'description': 'BERT с дополнительными фичами'
    })

In [ ]:
def analyze_and_plot_results(results_df):
    """Анализ и визуализация результатов экспериментов"""
    
    # Группируем по типу эксперимента
    results_df['experiment_type'] = results_df['experiment'].apply(
        lambda x: 'feature_engineering' if 'feature' in x else 
                  'model' if any(m in x for m in ['Logistic', 'RandomForest', 'XGBoost', 'LightGBM', 'CatBoost']) else
                  'bert' if 'bert' in x else 'ensemble'
    )
    
    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. ROC-AUC по типам экспериментов
    sns.boxplot(x='experiment_type', y='roc_auc', data=results_df, ax=axes[0, 0])
    axes[0, 0].set_title('ROC-AUC по типам экспериментов')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Лучшие модели
    best_results = results_df.nlargest(10, 'roc_auc')
    sns.barplot(x='roc_auc', y='experiment', data=best_results, ax=axes[0, 1])
    axes[0, 1].set_title('Топ-10 экспериментов по ROC-AUC')
    
    # 3. Время обучения vs качество
    if 'fit_time' in results_df.columns:
        sns.scatterplot(x='fit_time', y='roc_auc', hue='experiment_type', 
                       data=results_df, ax=axes[1, 0], s=100)
        axes[1, 0].set_title('Время обучения vs ROC-AUC')
    
    # 4. Стабильность результатов (std)
    if 'std' in results_df.columns:
        sns.scatterplot(x='roc_auc', y='std', hue='experiment_type', 
                       data=results_df, ax=axes[1, 1], s=100)
        axes[1, 1].set_title('ROC-AUC vs Стандартное отклонение')
    
    plt.tight_layout()
    plt.savefig('comprehensive_experiment_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Статистика
    for exp_type in results_df['experiment_type'].unique():
        subset = results_df[results_df['experiment_type'] == exp_type]
        print(f"{exp_type}: {len(subset)} экспериментов, "
              f"ROC-AUC: {subset['roc_auc'].mean():.4f} ± {subset['roc_auc'].std():.4f}")
    
    # Лучший эксперимент
    best_exp = results_df.loc[results_df['roc_auc'].idxmax()]
    print(f"\nЛучший эксперимент: {best_exp['experiment']}")
    print(f"ROC-AUC: {best_exp['roc_auc']:.4f}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Оценка баллов за этот пункт будет делиться на две части, 

- качество проведенных эксперементов максимум один балл 

- баллы за соревнование 2 * (1 - (Ваше_место - 1)/(количество_участников - 1))